In [1]:
import gradio as gr
import requests

In [ ]:
import requests
import uuid

API_KEY = '***'
ENDPOINT = "https://6b039-language2.cognitiveservices.azure.com/language/:analyze-text?api-version=2022-05-01"

HEADER = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": API_KEY
}


def build_body(kind, text, language=None):
    request_id = str(uuid.uuid4())
    body = {
        "kind": kind,
        "parameters": {
            "modelVersion": "latest"
        },
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "text": text
                }
            ]
        }
    }
    if language:
        body['analysisInput']['documents'][0]['language'] = language
    
    return body


def detect_language(text=""):
    response = requests.post(ENDPOINT, headers=HEADER, json=build_body('LanguageDetection', text))
    if response.status_code == 200:
        response_json = response.json()
        language_code = response_json["results"]["documents"][0]["detectedLanguage"]["iso6391Name"]
        return language_code
    else:
        return response.status_code, response.reason
    

def recognize_named_entity(text=""):
    language = detect_language(text)
    response = requests.post(ENDPOINT, headers=HEADER, json=build_body('EntityRecognition', text, language))
    
    if response.status_code == 200:
        response_json = response.json()
        results = []
        for entity in response_json['results']['documents'][0]['entities']:
            entity_description = f'{entity['category']}/'
            if 'subcategory' in entity:
                entity_description += f'{entity['subcategory']}/'
            entity_description = f'{entity_description:60}\t{entity['text']}\t{entity['confidenceScore'] * 100:.2f} %'
            results.append(entity_description)
        return '\n'.join(results)
    else:
        return response.status_code, response.reason
    

def recognize_entity_pii(text=""):
    language = detect_language(text)
    response = requests.post(ENDPOINT, headers=HEADER, json=build_body('PiiEntityRecognition', text, language))
    
    if response.status_code == 200:
        response_json = response.json()
        results = []
        for entity in response_json['results']['documents'][0]['entities']:
            entity_description = f'{entity['category']}/'
            if 'subcategory' in entity:
                entity_description += f'{entity['subcategory']}/'
            entity_description = f'{entity_description:60}\t{entity['text']}\t{entity['confidenceScore'] * 100:.2f} %'
            results.append(entity_description)
        return '\n'.join(results)
    else:
        return response.status_code, response.reason

In [9]:
type_choices = ['Language Detection', 'Named Entity Recognization', 'PII Entity Recognization']

def perform_send(choice, text):
    text = text.strip()
    
    if not text:
        return
    
    if choice == type_choices[0]:
        return detect_language(text)
    elif choice == type_choices[1]:
        return recognize_named_entity(text)
    else:
        return recognize_entity_pii(text)
    

with gr.Blocks() as demo:
    gr.Markdown('# 🔠 자연어 분석')
    with gr.Row():
        with gr.Column():
            input_textbox = gr.Textbox(label='📤 요청 데이터', lines=10, max_lines=10)
            type_radio = gr.Radio(label='🔠 분석 유형', choices=type_choices, value=type_choices[0], interactive=True)
        with gr.Column():
            output_textbox = gr.Textbox(label='📥 응답 데이터', lines=16, max_lines=16, interactive=False)
    send_button = gr.Button('요청 보내기')
    send_button.click(perform_send, inputs=[type_radio, input_textbox], outputs=[output_textbox])

demo.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
